<a href="https://colab.research.google.com/github/JashVaghasiya/EasyMinutes-MOM/blob/main/Action_words_Rephraser.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
pip install transformers nltk torach numpy pandas

ERROR: Could not find a version that satisfies the requirement torach (from versions: none)
ERROR: No matching distribution found for torach


In [ ]:
pip install spacy download en_core_web_sm

In [ ]:
import gc
from google.colab import drive

drive.mount('/content/drive')
gc.collect()

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


120

In [ ]:
from transformers import BertTokenizer, BertForSequenceClassification, T5ForConditionalGeneration, T5Tokenizer
import torch
import nltk
import spacy

In [ ]:
# Load BERT models and tokenizer for action detection
folder_path = '/content/drive/My Drive/my_model'
bert_tokenizer = BertTokenizer.from_pretrained(folder_path)
bert_model = BertForSequenceClassification.from_pretrained(folder_path)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
bert_model.to(device)

BertForSequenceClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0-11): 12 x BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12,

In [ ]:
# Load T5 model and tokenizer for paraphrasing
t5_tokenizer = T5Tokenizer.from_pretrained("t5-small")
t5_model = T5ForConditionalGeneration.from_pretrained("t5-small")
t5_model.to(device)

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


T5ForConditionalGeneration(
  (shared): Embedding(32128, 512)
  (encoder): T5Stack(
    (embed_tokens): Embedding(32128, 512)
    (block): ModuleList(
      (0): T5Block(
        (layer): ModuleList(
          (0): T5LayerSelfAttention(
            (SelfAttention): T5Attention(
              (q): Linear(in_features=512, out_features=512, bias=False)
              (k): Linear(in_features=512, out_features=512, bias=False)
              (v): Linear(in_features=512, out_features=512, bias=False)
              (o): Linear(in_features=512, out_features=512, bias=False)
              (relative_attention_bias): Embedding(32, 8)
            )
            (layer_norm): T5LayerNorm()
            (dropout): Dropout(p=0.1, inplace=False)
          )
          (1): T5LayerFF(
            (DenseReluDense): T5DenseActDense(
              (wi): Linear(in_features=512, out_features=2048, bias=False)
              (wo): Linear(in_features=2048, out_features=512, bias=False)
              (dropout): Drop

In [ ]:
# Download necessary NLTK models and corpora
nltk.download('punkt', quiet=True)
nltk.download('averaged_perceptron_tagger', quiet=True)
nltk.download('maxent_ne_chunker', quiet=True)
nltk.download('words', quiet=True)

True

In [ ]:
# Function to split transcript into sentences
def split_into_sentences(transcript):
    sentences = nltk.tokenize.sent_tokenize(transcript)
    return sentences

In [ ]:
# Function to rephrase action sentences using T5
def rephrase_with_model(sentence):
    input_ids = t5_tokenizer.encode(sentence, return_tensors="pt").to(device)
    outputs = t5_model.generate(input_ids, max_length=100, num_beams=5, early_stopping=True)
    paraphrased_text = t5_tokenizer.decode(outputs[0], skip_special_tokens=True)
    return paraphrased_text

In [ ]:
nlp = spacy.load("en_core_web_sm")

# Function to extract action words (verbs) and create phrases
def extract_action_words_and_create_phrases(sentence):
    doc = nlp(sentence)
    phrases = []

    for token in doc:
        if token.pos_ == "VERB":
            # Start with the verb itself
            phrase = [token.text]

            # Capture adverbial modifiers of the verb (e.g., "quickly ran")
            adverbs = [child.text for child in token.children if child.dep_ == "advmod"]
            if adverbs:
                phrase = adverbs + phrase

            # Look for the verb's direct object, prepositional phrases, or nominal subjects
            for child in token.children:
                if child.dep_ in ["dobj", "pobj", "nsubj", "nsubjpass"]:
                    # Include compound nouns or adjectives modifying the noun
                    modifiers = [grandchild.text for grandchild in child.children if grandchild.dep_ in ["compound", "amod"]]
                    full_noun = ' '.join(modifiers + [child.text])
                    phrase.append(full_noun)
                elif child.dep_ == "prep":
                    # Include the preposition and its object, with modifiers
                    prep_phrase = [child.text]
                    for grandchild in child.children:
                        if grandchild.dep_ == "pobj":
                            prep_modifiers = [ggc.text for ggc in grandchild.children if ggc.dep_ in ["compound", "amod"]]
                            full_prep_noun = ' '.join(prep_modifiers + [grandchild.text])
                            prep_phrase.append(full_prep_noun)
                    phrase.append(' '.join(prep_phrase))

            # Reconstruct the phrase while preserving the original order as much as possible
            phrases.append(' '.join(phrase))

    return phrases

In [ ]:
# Function to classify, rephrase sentences, and extract action phrases
def process_transcript_and_extract_phrases(transcript):
    sentences = split_into_sentences(transcript)
    action_phrases = []

    for sentence in sentences:
        inputs = bert_tokenizer(sentence, padding=True, truncation=True, max_length=128, return_tensors="pt")
        inputs = {key: value.to(device) for key, value in inputs.items()}

        with torch.no_grad():
            outputs = bert_model(**inputs)

        probabilities = torch.nn.functional.softmax(outputs.logits, dim=-1)
        predicted_class = torch.argmax(probabilities, dim=-1).item()

        # If the sentence is classified as 'Action'
        if predicted_class == 1:
            action_words_phrases = extract_action_words_and_create_phrases(sentence)
            # rephrased_words = rephrase_with_model(action_words_phrases)
            action_phrases.extend(action_words_phrases)

    return action_phrases

In [ ]:
# Example usage
transcript = "Your transcript text here. Make sure to replace this with your actual transcript."
action_phrases = process_transcript_and_extract_phrases(transcript)
print("Action phrases:", action_phrases)

Action phrases: ['Make', 'replace this with actual transcript']


### Method 2

In [ ]:
import spacy
import torch
from transformers import BertTokenizer, BertForSequenceClassification, T5Tokenizer, T5ForConditionalGeneration
import nltk

# Make sure to download necessary NLTK data and spaCy model
nltk.download('punkt', quiet=True)
# !python -m spacy download en_core_web_sm

# Load spaCy for NLP tasks
nlp = spacy.load("en_core_web_sm")

# Load BERT and T5 models and tokenizers
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Assuming 'folder_path' contains the BERT model and tokenizer
folder_path = '/content/drive/MyDrive/my_model'  # Update this path
bert_tokenizer = BertTokenizer.from_pretrained(folder_path)
bert_model = BertForSequenceClassification.from_pretrained(folder_path).to(device)

t5_tokenizer = T5Tokenizer.from_pretrained("t5-small")
t5_model = T5ForConditionalGeneration.from_pretrained("t5-small").to(device)

def split_into_sentences(transcript):
    """Split transcript into sentences using NLTK."""
    return nltk.tokenize.sent_tokenize(transcript)

def rephrase_with_model(sentence, model, tokenizer, device):
    """Rephrase a sentence using the T5 model."""
    input_ids = tokenizer.encode(sentence, return_tensors="pt").to(device)
    outputs = model.generate(input_ids, max_length=100, num_beams=5, early_stopping=True)
    return tokenizer.decode(outputs[0], skip_special_tokens=True)

def extract_and_rephrase_action_phrases(sentence, nlp, t5_model, t5_tokenizer, device):
    """Extract action phrases from a sentence and rephrase them for clarity."""
    doc = nlp(sentence)
    action_phrase = ""

    for token in doc:
        # Check for the verb that indicates an action
        if token.pos_ == "VERB":
            # Construct a basic phrase: Verb + Direct Object
            direct_objects = [child for child in token.children if child.dep_ == "dobj"]
            if direct_objects:
                action_phrase = token.text + " " + " ".join([dobj.text for dobj in direct_objects])
            else:
                # If no direct object, use the verb itself
                action_phrase = token.text
            break  # Focus on the first action verb found for simplicity

    # Rephrase the constructed action phrase for better clarity, if any phrase was constructed
    if action_phrase:
        # Correcting the model usage with proper input formatting
        input_ids = t5_tokenizer.encode(action_phrase, return_tensors="pt").to(device)
        outputs = t5_model.generate(input_ids, max_length=50, num_beams=5, early_stopping=True)
        rephrased_phrase = t5_tokenizer.decode(outputs[0], skip_special_tokens=True)
        return rephrased_phrase
    else:
        return ""  # Return an empty string if no action verb was identified or no phrase could be constructed

def process_transcript_and_extract_refined_phrases(transcript, bert_model, bert_tokenizer, t5_model, t5_tokenizer, nlp, device):
    sentences = split_into_sentences(transcript)
    refined_phrases = []

    for sentence in sentences:
        inputs = bert_tokenizer.encode_plus(sentence, return_tensors="pt").to(device)
        outputs = bert_model(**inputs)  # Corrected line: inputs are unpacked as a dictionary
        probabilities = torch.nn.functional.softmax(outputs.logits, dim=-1)
        predicted_class = torch.argmax(probabilities, dim=-1).item()

        if predicted_class == 1:  # Action sentence
            refined_phrase = extract_and_rephrase_action_phrases(sentence, nlp, t5_model, t5_tokenizer, device)
            if refined_phrase:
                refined_phrases.append(refined_phrase)

    return refined_phrases

# Example usage
transcript = "you're likely here because you're searching for a random sentence. Sometimes a random word just isn't enough, and that is where the random sentence generator comes into play. By inputting the desired number, you can make a list of as many random sentences as you want or need. Producing random sentences can be helpful in a number of different ways."
refined_action_phrases = process_transcript_and_extract_refined_phrases(
    transcript, bert_model, bert_tokenizer, t5_model, t5_tokenizer, nlp, device)
print("Refined Action Phrases:", refined_action_phrases)


Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


Refined Action Phrases: ['in putting number', 'Produc sentences']
